Read parameters from file

In [14]:
from cfdmod.use_cases.roughness_gen.parameters import PositionParams
import pathlib

import pprint

from lnas import LnasFormat
import trimesh

import numpy as np

from cfdmod.use_cases.roughness_gen.parameters import GenerationParams

from cfdmod.use_cases.roughness_gen import build_single_element, linear_pattern


from cfdmod.api.geometry.STL import export_stl


pp = pprint.PrettyPrinter()

cases = ["000", "090", "180", "270"]  # s1
end_of_blocks = [86, 82, 94, 94]
# cases=['000', '045', '090', '135', '180', '225', '270', '315'] #cp

surfaces_names = ["terrain"]

cases_root_dir = pathlib.Path(
    "/mnt/disk01/prd-eng/cases/s1_consulting_nassu_test/025_BrookfieldAlceu/"
)

cfg_file = cases_root_dir / "simulation_data/configs_cfdmod/roughness_params_s24.yaml"
cfg = PositionParams.from_file(cfg_file)

output_path_root = cases_root_dir / "setup/STLs/STLs_files/"

results_path = cases_root_dir / "results/pilot/"
bodies_relative_path = pathlib.Path("000/bodies/lnas")
pp.pprint(cfg.model_dump())

{'bounding_box': {'end': (74.0, 128.0, 160.0), 'start': (1.0, 32.0, 0.0)},
 'element_params': {'height': 0.0833333333333333, 'width': 0.25},
 'spacing_params': {'line_offset': 0.791666666666667,
                    'offset_direction': 'y',
                    'spacing': (0.666666666666667, 1.33333333333333)},
 'surfaces': {'downwind_loft': '/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/results/000/000/bodies/lnas/downwind_loft.transformed.lnas',
              'terrain': '/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/results/000/000/bodies/lnas/terrain.transformed.lnas',
              'upwind_loft': '/mnt/disk01/prd-eng/consulting/030_MontoPirelliCampinas/results/000/000/bodies/lnas/upwind_loft.transformed.lnas'}}


In [15]:
for case, end_blocks in zip(cases, end_of_blocks):
    # overwrite surfaces list to be compatible with current case
    cfg.surfaces = {}
    for surface in surfaces_names:
        cfg.surfaces[surface] = (results_path / case) / (
            bodies_relative_path / f"{surface}.transformed.lnas"
        )

    end = list(cfg.bounding_box.end)
    cfg.bounding_box.end = (end_blocks, end[1], end[2])
    print(cfg.bounding_box.end)

    # Read mesh files and get surfaces bounding box
    surfaces_read: dict[str, LnasFormat] = {}
    surfaces_mesh: dict[str, trimesh.Trimesh] = {}

    mesh_bbox = [
        (float("inf"), float("inf"), float("inf")),
        (float("-inf"), float("-inf"), float("-inf")),
    ]

    for sfc, sfc_path in cfg.surfaces.items():
        lnas = LnasFormat.from_file(pathlib.Path(sfc_path))
        surfaces_read[sfc] = lnas
        surfaces_mesh[sfc] = trimesh.Trimesh(
            vertices=lnas.geometry.vertices, faces=lnas.geometry.triangles
        )

        min_point = surfaces_read[sfc].geometry.vertices.min(axis=0)
        max_point = surfaces_read[sfc].geometry.vertices.max(axis=0)

        if any(min_point < mesh_bbox[0]):
            mesh_bbox[0] = np.array([min_point, mesh_bbox[0]]).min(axis=0)
        if any(max_point > mesh_bbox[1]):
            mesh_bbox[1] = np.array([max_point, mesh_bbox[1]]).max(axis=0)

    s = np.array([cfg.bounding_box.start, mesh_bbox[0]]).max(axis=0)
    e = np.array([cfg.bounding_box.end, mesh_bbox[1]]).min(axis=0)
    bbox_to_use = np.array([s, e])

    # Calculate generation parameters
    lx = bbox_to_use[1][0] - bbox_to_use[0][0]
    ly = bbox_to_use[1][1] - bbox_to_use[0][1]

    if cfg.spacing_params.offset_direction == "x":
        Nx = int((lx - cfg.spacing_params.line_offset) // (cfg.spacing_params.spacing[0]) + 1)
        Ny = int(
            (ly + cfg.spacing_params.spacing[1])
            // (cfg.element_params.width + cfg.spacing_params.spacing[1])
        )
    else:
        Nx = int(lx // (cfg.spacing_params.spacing[0]) + 1)
        Ny = int(
            (ly + cfg.spacing_params.spacing[1] - cfg.spacing_params.line_offset)
            // (cfg.element_params.width + cfg.spacing_params.spacing[1])
        )

    # Setup generation parameters
    generation_params = GenerationParams(
        N_elements_x=Nx,
        N_elements_y=Ny,
        element_params=cfg.element_params,
        spacing_params=cfg.spacing_params,
    )

    # Build elements and apply linear patterns

    triangles, normals = build_single_element(generation_params.element_params)

    single_line_triangles, single_line_normals = linear_pattern(
        triangles,
        normals,
        direction=generation_params.spacing_params.offset_direction,
        n_repeats=generation_params.single_line_elements,
        spacing_value=generation_params.single_line_spacing,
    )

    full_triangles, full_normals = linear_pattern(
        single_line_triangles,
        single_line_normals,
        direction=generation_params.perpendicular_direction,
        n_repeats=generation_params.multi_line_elements,
        spacing_value=generation_params.multi_line_spacing,
        offset_value=generation_params.spacing_params.line_offset,
    )

    # Offset to match bounding box limits
    full_triangles[:, :, 0] += bbox_to_use[0][0]
    full_triangles[:, :, 1] += bbox_to_use[0][1]

    print(
        "Single element:",
        f"Vertices count: {triangles.shape[0] * triangles.shape[1]}",
        f"Triangles count: {len(triangles)}",
    )
    print(
        "Single line",
        f"Vertices count: {single_line_triangles.shape[0] * single_line_triangles.shape[1]}",
        f"Triangles count: {len(single_line_triangles)}",
    )
    print(
        "Replicated lines",
        f"Vertices count: {full_triangles.shape[0] * full_triangles.shape[1]}",
        f"Triangles count: {len(full_triangles)}",
    )

    # Create profiles with x normal planes
    x_vals = np.unique(full_triangles[:, :, 0].reshape(-1))

    profiles: dict[float, np.ndarray] = {}

    for sfc_label, mesh in surfaces_mesh.items():
        x_vals_to_use = x_vals[
            (x_vals >= mesh.vertices[:, 0].min()) & (x_vals <= mesh.vertices[:, 0].max())
        ]

        for x_val in x_vals_to_use:
            section_slice = mesh.section(
                plane_origin=[x_val, 0, 0],
                plane_normal=[-1, 0, 0],
            )
            vertices = np.array(section_slice.to_dict()["vertices"])
            if x_val in profiles.keys():
                profiles[x_val] = np.concatenate((profiles[x_val], vertices))
            else:
                profiles[x_val] = vertices

    for k, profile in profiles.items():
        profile = np.array(sorted(profile, key=lambda x: x[1]))

    # Offset geometry with surface information
    for tri in full_triangles:
        left_bound = tri[:, 1].min()
        right_bound = tri[:, 1].max()
        prof = profiles[tri[0][0]][
            (profiles[tri[0][0]][:, 1] >= left_bound) & (profiles[tri[0][0]][:, 1] <= right_bound)
        ]
        while len(prof) == 0:
            value_offset = generation_params.element_params.width
            left_bound -= value_offset
            right_bound += value_offset
            prof = profiles[tri[0][0]][
                (profiles[tri[0][0]][:, 1] >= left_bound)
                & (profiles[tri[0][0]][:, 1] <= right_bound)
            ]

        offset_val = prof[:, 2].min()
        tri[:, 2] += offset_val

    # Export generated geometry
    export_stl((output_path_root / case) / "positioned_elements.stl", full_triangles, full_normals)

(86, 128.0, 160.0)
Single element: Vertices count: 6 Triangles count: 2
Single line Vertices count: 360 Triangles count: 120
Replicated lines Vertices count: 46080 Triangles count: 15360
(82, 128.0, 160.0)
Single element: Vertices count: 6 Triangles count: 2
Single line Vertices count: 360 Triangles count: 120
Replicated lines Vertices count: 43920 Triangles count: 14640
(94, 128.0, 160.0)
Single element: Vertices count: 6 Triangles count: 2
Single line Vertices count: 360 Triangles count: 120
Replicated lines Vertices count: 50400 Triangles count: 16800
(94, 128.0, 160.0)
Single element: Vertices count: 6 Triangles count: 2
Single line Vertices count: 360 Triangles count: 120
Replicated lines Vertices count: 50400 Triangles count: 16800
